In [1]:
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd
from datetime import datetime

from __future__ import print_function
import pickle

In [2]:
url = 'https://www.the-numbers.com/movie/budgets/all/801'

user_agent = {'User-agent': 'Mozilla/5.0'}

response  = requests.get(url, headers = user_agent)

page = response.text

soup = BeautifulSoup(page,"lxml")

In [3]:
table = soup.find('table')
table

<table>
<tr><th> </th><th>Release<br/>Date</th><th>Movie</th><th>Production<br/>Budget</th><th>Domestic<br/>Gross</th><th>Worldwide<br/>Gross</th></tr>
<tr><td class="data">801</td>
<td><a href="/box-office-chart/daily/2020/08/26">Aug 26, 2020</a></td>
<td><b><a href="/movie/New-Mutants-The-(2020)#tab=summary">The New Mutants</a></b></td>
<td class="data"> $67,000,000</td>
<td class="data"> $23,855,569</td>
<td class="data"> $44,616,597</td>
</tr>
<tr><td class="data">802</td>
<td><a href="/box-office-chart/daily/1996/09/20">Sep 20, 1996</a></td>
<td><b><a href="/movie/Last-Man-Standing#tab=summary">Last Man Standing</a></b></td>
<td class="data"> $67,000,000</td>
<td class="data"> $18,115,927</td>
<td class="data"> $18,115,927</td>
</tr>
<tr><td class="data">803</td>
<td><a href="/box-office-chart/daily/2007/08/17">Aug 17, 2007</a></td>
<td><b><a href="/movie/Last-Legion-The#tab=summary">The Last Legion</a></b></td>
<td class="data"> $67,000,000</td>
<td class="data"> $5,932,060</td>


In [4]:
rows = [row for row in table.find_all('tr')]

In [5]:
movies = {}

for row in rows[1:]:
    items = row.find_all('td')
    link = row.find('a')
    title, url = link.text, link['href']
    movies[title] = [url] + [i.text for i in items]
    
movies

{'Aug 26, 2020': ['/box-office-chart/daily/2020/08/26',
  '801',
  'Aug 26, 2020',
  'The New Mutants',
  '\xa0$67,000,000',
  '\xa0$23,855,569',
  '\xa0$44,616,597'],
 'Sep 20, 1996': ['/box-office-chart/daily/1996/09/20',
  '802',
  'Sep 20, 1996',
  'Last Man Standing',
  '\xa0$67,000,000',
  '\xa0$18,115,927',
  '\xa0$18,115,927'],
 'Aug 17, 2007': ['/box-office-chart/daily/2007/08/17',
  '803',
  'Aug 17, 2007',
  'The Last Legion',
  '\xa0$67,000,000',
  '\xa0$5,932,060',
  '\xa0$25,357,771'],
 'Apr 30, 1999': ['/box-office-chart/daily/1999/04/30',
  '804',
  'Apr 30, 1999',
  'Entrapment',
  '\xa0$66,000,000',
  '\xa0$87,707,396',
  '\xa0$211,700,000'],
 'Jun 19, 1998': ['/box-office-chart/daily/1998/06/19',
  '805',
  'Jun 19, 1998',
  'The X Files: Fight the Future',
  '\xa0$66,000,000',
  '\xa0$83,898,313',
  '\xa0$189,176,423'],
 'Mar 13, 2014': ['/box-office-chart/daily/2014/03/13',
  '806',
  'Mar 13, 2014',
  'Need for Speed',
  '\xa0$66,000,000',
  '\xa0$43,568,507',
  '

In [6]:
budget = pd.DataFrame(movies).T  #transpose
budget.columns = ['link_stub','rank_all_movies','release','title', 
                    'budget', 'domestic_gross','worldwide_gross']

budget.shape

(98, 7)

In [7]:
budget['budget'] = budget['budget'].str.replace('$', '')
budget['budget'] = budget['budget'].str.replace(',', '')
budget['budget'] = budget['budget'].astype(int)

In [8]:
budget.head()

,link_stub,rank_all_movies,release,title,budget,domestic_gross,worldwide_gross
"Aug 26, 2020",/box-office-chart/daily/2020/08/26,801,"Aug 26, 2020",The New Mutants,67000000,"$23,855,569","$44,616,597"
"Sep 20, 1996",/box-office-chart/daily/1996/09/20,802,"Sep 20, 1996",Last Man Standing,67000000,"$18,115,927","$18,115,927"
"Aug 17, 2007",/box-office-chart/daily/2007/08/17,803,"Aug 17, 2007",The Last Legion,67000000,"$5,932,060","$25,357,771"
"Apr 30, 1999",/box-office-chart/daily/1999/04/30,804,"Apr 30, 1999",Entrapment,66000000,"$87,707,396","$211,700,000"
"Jun 19, 1998",/box-office-chart/daily/1998/06/19,805,"Jun 19, 1998",The X Files: Fight the Future,66000000,"$83,898,313","$189,176,423"


In [9]:
budget.to_csv('budget9.csv')